In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import class_weight
import numpy as np
import tensorflow_quantum as tfq
import cirq

# Set up directories for training and testing
train_dir = '/kaggle/input/parkinsons-drawings/'
test_dir = '/kaggle/input/parkinsons-drawings/'

# Data Preprocessing and Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load and iterate the dataset
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Load pre-trained VGG16 model without the top layers
vgg16_base = tf.keras.applications.VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)

# Freeze all layers in the VGG16 base model
vgg16_base.trainable = False

# Add new classifier layers on top
model = models.Sequential([
    vgg16_base,
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Adding a quantum layer
# Create a quantum circuit
def create_quantum_circuit():
    qubits = cirq.LineQubit.range(2)  # Define qubits
    circuit = cirq.Circuit()
    circuit.append(cirq.H(qubits[0]))  # Apply Hadamard gate
    circuit.append(cirq.CNOT(qubits[0], qubits[1]))  # Apply CNOT gate
    return circuit

# Quantum model layer for QML
class QuantumLayer(layers.Layer):
    def __init__(self, **kwargs):
        super(QuantumLayer, self).__init__(**kwargs)
        self.qubits = cirq.LineQubit.range(2)
        self.circuit = create_quantum_circuit()

    def call(self, inputs):
        # Here, we would integrate quantum operations
        # using TensorFlow Quantum's support
        circuit = self.circuit
        quantum_data = tfq.convert_to_tensor([circuit])
        return tfq.layers.PQC(circuit, self.qubits)(quantum_data)

# Add the quantum layer to the model
model.add(QuantumLayer())

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Compute class weights to handle class imbalance
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))

# Fine-tune some layers of VGG16
for layer in vgg16_base.layers[-4:]:
    layer.trainable = True

# Use EarlyStopping to stop training when validation loss stops improving
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=30,  # Increased epochs
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size,
    class_weight=class_weights,
    callbacks=[early_stopping]
)

# Evaluate the model on the test dataset
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test Accuracy: {test_acc * 100:.2f}%')

# Save the model
model.save('vgg16_parkinson_classifier_with_qml.h5')


TypeError: DaskFinder.find_spec() got an unexpected keyword argument 'path'

In [13]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

import tensorflow_quantum as tfq
print("TensorFlow Quantum version:", tfq.__version__)

import cirq
print("Cirq version:", cirq.__version__)


TensorFlow version: 2.18.0


TypeError: DaskFinder.find_spec() got an unexpected keyword argument 'path'

In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import class_weight
import numpy as np
import tensorflow_quantum as tfq
import cirq

# Set up directories for training and testing
train_dir = '/kaggle/input/parkinsons-drawings/spiral/training'
test_dir = '/kaggle/input/parkinsons-drawings/spiral/testing'

# Data Preprocessing and Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load and iterate the dataset
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Load pre-trained VGG16 model without the top layers
vgg16_base = tf.keras.applications.VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)

# Freeze all layers in the VGG16 base model
vgg16_base.trainable = False

# Add new classifier layers on top
model = models.Sequential([
    vgg16_base,
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Adding a simple quantum layer to the model
def create_quantum_circuit():
    # Create a quantum circuit with two qubits
    qubits = cirq.LineQubit.range(2)
    circuit = cirq.Circuit()
    circuit.append(cirq.H(qubits[0]))  # Apply Hadamard gate to qubit 0
    circuit.append(cirq.CNOT(qubits[0], qubits[1]))  # Apply CNOT gate between qubits
    return circuit

# Quantum Layer class
class QuantumLayer(layers.Layer):
    def __init__(self, **kwargs):
        super(QuantumLayer, self).__init__(**kwargs)
        self.qubits = cirq.LineQubit.range(2)  # Create 2 qubits for the quantum circuit
        self.circuit = create_quantum_circuit()

    def call(self, inputs):
        # Convert the classical inputs to quantum format
        quantum_data = tfq.convert_to_tensor([self.circuit])
        return tfq.layers.PQC(self.circuit, self.qubits)(quantum_data)

# Add the quantum layer to the model
model.add(QuantumLayer())

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Compute class weights to handle class imbalance
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))

# Fine-tune some layers of VGG16
for layer in vgg16_base.layers[-4:]:
    layer.trainable = True

# Use EarlyStopping to stop training when validation loss stops improving
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=30,  # Increased epochs
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size,
    class_weight=class_weights,
    callbacks=[early_stopping]
)

# Evaluate the model on the test dataset
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test Accuracy: {test_acc * 100:.2f}%')

# Save the model
model.save('vgg16_parkinson_classifier_with_qml.h5')


TypeError: DaskFinder.find_spec() got an unexpected keyword argument 'path'

In [11]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import class_weight
import numpy as np

# Set up directories for training and testing
train_dir = '/kaggle/input/parkinsons-drawings/spiral/training'
test_dir = '/kaggle/input/parkinsons-drawings/spiral/testing'

# Data Preprocessing and Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load and iterate the dataset
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Load pre-trained VGG16 model without the top layers
vgg16_base = tf.keras.applications.VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)

# Freeze all layers in the VGG16 base model
vgg16_base.trainable = False

# Add new classifier layers on top
model = models.Sequential([
    vgg16_base,
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Compute class weights to handle class imbalance
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))

# Fine-tune some layers of VGG16
for layer in vgg16_base.layers[-4:]:
    layer.trainable = True

# Use EarlyStopping to stop training when validation loss stops improving
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=30,  # Increased epochs
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size,
    class_weight=class_weights,
    callbacks=[early_stopping]
)

# Evaluate the model on the test dataset
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test Accuracy: {test_acc * 100:.2f}%')

# Save the model
model.save('vgg16_parkinson_classifier_with_qml.h5')


Found 72 images belonging to 2 classes.
Found 30 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 22s 17s/step - accuracy: 0.4583 - loss: 0.7947 - val_accuracy: 0.6000 - val_loss: 0.8104
Epoch 2/30
1/2 ━━━━━━━━━━━━━━━━━━━━ 9s 10s/step - accuracy: 0.6250 - loss: 0.5086

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 391ms/step - accuracy: 0.6250 - loss: 0.5086 - val_accuracy: 0.6333 - val_loss: 0.8408
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.6167 - loss: 0.6253 - val_accuracy: 0.7667 - val_loss: 0.4653
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 382ms/step - accuracy: 0.6875 - loss: 0.6464 - val_accuracy: 0.8333 - val_loss: 0.4860
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 466ms/step - accuracy: 0.5437 - loss: 0.7459 - val_accuracy: 0.7667 - val_loss: 0.4675
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 416ms/step - accuracy: 0.7500 - loss: 0.4794 - val_accuracy: 0.8000 - val_loss: 0.4635
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 485ms/step - accuracy: 0.7125 - loss: 0.5453 - val_accuracy: 0.7333 - val_loss: 0.5003
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 384ms/step - accuracy: 0.8750 - loss: 0.3821 - val_accuracy: 0.7333 - val_loss: 0.5395
Epoch 9/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 771ms/step - accuracy: 0.6979 - loss: 0.5486 - val_accuracy: 0.7667 - val_loss: 0.4679
Epoch